In [1]:
import pandas as pd
import plotly.express as px
import dash
from dash import dcc, html
from dash.dependencies import Input, Output

In [2]:
# Load the data
df_2022 = pd.read_csv('Cleaneddata2022.csv')
df_2023 = pd.read_csv('Cleaneddata2023.csv')

In [3]:
# Combine the datasets
df = pd.concat([df_2022, df_2023], ignore_index=True)

In [4]:
# Convert columns to appropriate types
df['price'] = pd.to_numeric(df['price'], errors='coerce')
df['space'] = pd.to_numeric(df['space'], errors='coerce')
df['Price_per_square_meter'] = pd.to_numeric(df['Price_per_square_meter'], errors='coerce')

In [5]:
# Clean the data
df = df.dropna(subset=['Price_per_square_meter', 'space', 'city', 'area'])
df = df[(df['Price_per_square_meter'] > 0) & (df['space'] > 0)]

In [6]:
# Create the Dash app
app = dash.Dash(__name__)

In [7]:
# Define the layout
app.layout = html.Div([
    html.H1("Real Estate Market Analysis - Saudi Arabia", style={'textAlign': 'center'}),
    
    html.Div([
        html.Div([
            html.Label("Filter by Area:"),
            dcc.Dropdown(
                id='area-filter',
                options=[{'label': area, 'value': area} for area in sorted(df['area'].unique())],
                value=['Riyadh', 'Makkah', 'Eastern Province'],
                multi=True
            ),
        ], style={'width': '30%', 'display': 'inline-block', 'padding': '10px'}),
        
        html.Div([
            html.Label("Filter by City:"),
            dcc.Dropdown(
                id='city-filter',
                options=[{'label': city, 'value': city} for city in sorted(df['city'].unique())],
                value=sorted(df['city'].unique()),
                multi=True
            ),
        ], style={'width': '30%', 'display': 'inline-block', 'padding': '10px'}),
        
        html.Div([
            html.Label("Filter by Property Type:"),
            dcc.Dropdown(
                id='property-type-filter',
                options=[{'label': pt, 'value': pt} for pt in sorted(df['property_type'].unique())],
                value=sorted(df['property_type'].unique()),
                multi=True
            ),
        ], style={'width': '30%', 'display': 'inline-block', 'padding': '10px'}),
    ], style={'display': 'flex', 'justifyContent': 'space-between'}),
    
    # Insight 1: Graph (Average Price per Square Meter by City)
    html.Div([
        html.H2("Insight 1: Average Price per Square Meter by City", style={'textAlign': 'center'}),
        dcc.Graph(id='price-per-sqm-graph'),
    ]),
])

# Define the callback for updating the graphs
@app.callback(
    Output('price-per-sqm-graph', 'figure'),
    [Input('area-filter', 'value'),
     Input('city-filter', 'value'),
     Input('property-type-filter', 'value')]
)
def update_graphs(selected_areas, selected_cities, selected_property_types):
    # Filter the data based on selections
    filtered_df = df[
        (df['area'].isin(selected_areas)) &
        (df['city'].isin(selected_cities)) &
        (df['property_type'].isin(selected_property_types))
    ]
    
    # For insight 1: Average price per square meter by city
    price_by_city = filtered_df.groupby('city')['Price_per_square_meter'].mean().reset_index()
    price_by_city = price_by_city.sort_values('Price_per_square_meter', ascending=False)
    
    # Add area information to each city
    city_area_map = df.drop_duplicates('city')[['city', 'area']].set_index('city').to_dict()['area']
    price_by_city['area'] = price_by_city['city'].map(city_area_map)

    # Create the figure for Insight 1
    fig1 = px.bar(
        price_by_city,
        x='city',
        y='Price_per_square_meter',
        color='area',
        title='Average Price per Square Meter by City',
        labels={
            'Price_per_square_meter': 'Average Price per Sq.m (SAR)', 
            'city': 'City',
            'area': 'Area'
        },
        template='plotly_white'
    )

    # Enhance layout
    fig1.update_layout(
        xaxis={'categoryorder': 'total descending'},
        yaxis_title_font=dict(size=14),
        xaxis_title_font=dict(size=14),
        legend_title_font=dict(size=12),
        title_font=dict(size=16),
    )

    return fig1

In [9]:
# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)